# Compute electric mix for all countries

DYNAMICAL allow to only compute the electric mix for all considered countries at the time. These matrices are intermediate results in the usual DYNAMICAL process. The functionality was added due to its potential usefulness. 

In technical words, a function `get_inverted_matrix` allow to retrieve the invert of technology matrix $(I-A)^{-1}$.

In [1]:
from dynamical.easy_use import get_inverted_matrix # Import the function
from parameter import Parameter # Import the parameter handler object

In [2]:
# Configure parameter to reduce the number of time steps
my_param = Parameter().from_excel("Spreadsheet_test.xlsx")
my_param.start="2017-02-01 12:00"
my_param.end="2017-02-01 14:00"
my_param.freq = "H"

In [3]:
mix = get_inverted_matrix(config=my_param)

In [4]:
mix[0]

,Mix_AT,Mix_CH,Mix_DE,Mix_FR,Mix_IT,Mix_Other,Biomass_AT,Fossil_Brown_coal/Lignite_AT,Fossil_Coal-derived_gas_AT,Fossil_Gas_AT,...,Hydro_Run-of-river_and_poundage_IT,Hydro_Water_Reservoir_IT,Marine_IT,Nuclear_IT,Other_fossil_IT,Other_renewable_IT,Solar_IT,Waste_IT,Wind_Offshore_IT,Wind_Onshore_IT
Mix_AT,1.000034,1.914997e-02,1.385938e-04,0.0,0.000637,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mix_CH,0.001787,1.003225e+00,7.260626e-03,0.0,0.033385,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mix_DE,0.224013,4.451191e-01,1.003222e+00,0.0,0.014813,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mix_FR,0.007310,2.377682e-01,3.155733e-02,1.0,0.053205,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mix_IT,0.004988,9.551584e-05,6.912757e-07,0.0,1.000003,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Other_renewable_IT,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Solar_IT,0.000390,7.458758e-06,5.398119e-08,0.0,0.078089,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Waste_IT,0.000005,9.666313e-08,6.995790e-10,0.0,0.001012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Wind_Offshore_IT,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Build function to condense and visualize data

In [5]:
import numpy as np # need numpy functions 
import pandas as pd # need pandas functions

In [6]:
def group_by_family(matrix):
    families = {'Fossil':['Fossil_Brown_coal/Lignite', 'Fossil_Coal-derived_gas',
                          'Fossil_Gas', 'Fossil_Hard_coal', 'Fossil_Oil', 'Fossil_Oil_shale',
                          'Fossil_Peat','Other_fossil'],
                'Nuclear':['Nuclear'],
                'Hydro':['Hydro_Pumped_Storage','Hydro_Run-of-river_and_poundage',
                         'Hydro_Water_Reservoir', 'Marine'],
                'Other Renwable':['Biomass', 'Geothermal', 'Other_renewable',
                                  'Solar', 'Waste', 'Wind_Offshore', 'Wind_Onshore'],
                'External imports':['Mix']}
    
    per_family = []
    for f in families:
        idx = [k for k in matrix.index if "_".join(k.split("_")[:-1]) in families[f]]
        per_family.append(pd.Series(matrix.loc[idx].sum(), name=f))
    return pd.concat(per_family, axis=1)


def group_per_country(matrix):
    """Function to group datasets per country"""
    countries = np.unique([k.split("_")[-1] for k in matrix.index])
    
    per_country = []
    for c in countries:
        idx = [k for k in matrix.index if k.split("_")[-1]==c]
        per_country.append(pd.Series(matrix.loc[idx].sum(), name=c))
        
    return pd.concat(per_country,axis=1)

## Visualize the origin per production type (columns) in each country (index)

In [7]:
group_by_family(mix[0].loc["Mix_Other":, :"Mix_IT"]) # 1st time step

,Fossil,Nuclear,Hydro,Other Renwable,External imports
Mix_AT,0.485924,0.030762,0.261834,0.097567,0.123913
Mix_CH,0.298574,0.414085,0.150002,0.134030,0.003309
Mix_DE,0.593146,0.136779,0.030051,0.237876,0.002149
Mix_FR,0.128053,0.713730,0.093241,0.064976,0.000000
Mix_IT,0.665790,0.046107,0.095778,0.192215,0.000110


In [8]:
group_by_family(mix[1].loc["Mix_Other":, :"Mix_IT"]) # 2nd time step

,Fossil,Nuclear,Hydro,Other Renwable,External imports
Mix_AT,0.504585,0.033245,0.234267,0.097192,0.130710
Mix_CH,0.293068,0.411462,0.165844,0.128263,0.001363
Mix_DE,0.594173,0.141458,0.028400,0.233368,0.002600
Mix_FR,0.131565,0.717227,0.086283,0.064926,0.000000
Mix_IT,0.664596,0.051813,0.110943,0.172590,0.000058


## Visualize the origin per country (columns) in each country (index)

In [9]:
group_per_country(mix[0].loc["Mix_Other":, :"Mix_IT"]) # 1st time step

,AT,CH,DE,FR,IT,Other
Mix_AT,0.648382,0.000568,0.215232,0.007310,4.596044e-03,0.123913
Mix_CH,0.012416,0.318747,0.427671,0.237768,8.801108e-05,0.003309
Mix_DE,0.000090,0.002307,0.963897,0.031557,6.369617e-07,0.002149
Mix_FR,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000
Mix_IT,0.000413,0.010607,0.014232,0.053205,9.214323e-01,0.000110


In [10]:
group_per_country(mix[1].loc["Mix_Other":, :"Mix_IT"]) # 2nd time step

,AT,CH,DE,FR,IT,Other
Mix_AT,0.636860,0.001400,0.218438,0.009143,3.448941e-03,0.130710
Mix_CH,0.001058,0.328765,0.422783,0.246025,5.729576e-06,0.001363
Mix_DE,0.000008,0.002599,0.957974,0.036818,4.530087e-08,0.002600
Mix_FR,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000
Mix_IT,0.000045,0.014004,0.018009,0.058283,9.096001e-01,0.000058
